### Imports

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pickle
import numpy as np
import matplotlib.pyplot as plt
from models.mog_model import *
from utils.distributions import *
from matplotlib import gridspec
from utils.plot_helper import errorbar_plot
plt.style.use('/Users/saforem2/.config/matplotlib/stylelib/dark_jupyter.mplstyle')

plt.rc('text', usetex=False)
plt.rcParams['errorbar.capsize'] = 0
%matplotlib notebook
%autoreload 2



#### Helper functions

In [ ]:
def pkl_loader(_dir, _file):
    with open(_dir + _file, 'rb') as f:
        _data = pickle.load(f)
    return _data

def load_data(data_dir):
    _d = {}
    data = {}
    for file in os.listdir(data_dir):
        if file.endswith('.pkl'):
            key = file[:-4]
            _d[key] = pkl_loader(data_dir, file)
        for key, val in _d.items():
            data[key] = val
    return data

In [ ]:
def fix_legends(axes):
    if isinstance(axes, (np.ndarray, list)):
        legends = [ax.get_legend() for ax in axes]
        for leg in legends:
            leg.texts[0].set_color('w')
    else:
        legend = axes.get_legend()
        for idx in range(len(legend.texts)):
            legend.texts[idx].set_color('w')
    return axes

In [ ]:
def add_vline(axes, x, **kwargs):
    if isinstance(axes, (np.ndarray, list)):
        for ax in axes:
            ax.axvline(x, **kwargs)
                       #, color='C3', ls=':', lw=2.)
    else:
        axes.axvline(x, **kwargs)
    return axes

In [ ]:
def fix_ticks(axes):
    if isinstance(axes, (np.ndarray, list)):
        for idx in range(len(axes)):
            axes[idx].tick_params(which='both', color='#474747', labelcolor='k')
    else:
        axes.tick_params(which='both', color='#474747', labelcolor='k')

### Define data_dir and load data

In [ ]:
run_dir = '../../log_mog_tf/run_376/'
data_dir = run_dir + 'run_info/'
figs_dir = run_dir + 'figures1/'
if not os.path.exists(figs_dir):
    os.makedirs(figs_dir)

In [ ]:
data = load_data(data_dir)

get_vals_as_arr = lambda _dict: np.array(list(_dict.values()))

tr0 = get_vals_as_arr(data['tunneling_rates'])
tr1 = get_vals_as_arr(data['tunneling_rates_highT'])
ar0 = get_vals_as_arr(data['acceptance_rates'])
ar1 = get_vals_as_arr(data['acceptance_rates_highT'])
d0 = get_vals_as_arr(data['distances'])
d1 = get_vals_as_arr(data['distances_highT'])

steps_arr = []
temp_arr = []
for key in data['tunneling_rates_highT'].keys():
    steps_arr.append(key[0]+1)
    temp_arr.append(key[1])

In [ ]:
x_steps = 3 * [steps_arr]
x_temps = 3 * [temp_arr]

y_data = [tr0[:, 0], ar0[:, 0], d0[:, 0]]
y_err = [tr0[:, 1], ar1[:, 1], d1[:, 1]]

y_data_highT = [tr1[:, 0], ar1[:, 0], d1[:, 0]]

y_err_highT = [tr1[:, 1], ar1[:, 1], d1[:, 1]]

str0 = (f"{data['_params']['num_distributions']} in {data['_params']['x_dim']} dims; ")
str1 = (r'$\mu_{ij} = \delta_{ij},' + r' \sigma = {{{0}}}$; '.format(data['_params']['sigma']))
#str1 = (r'$\mathcal{N}_{\hat \mu}(\1\hat \mu;$'
#        + r'${{{0}}}),$'.format(data['_params']['sigma']))
title = str0 + str1 + r'$T_{trajectory} = 1$'
title_highT = str0 + str1 + r'$T_{trajectory} > 1$'
def out_file(f): return figs_dir + f'{f}.pdf'

kwargs = {
    'fillstyle': 'full',
    'markersize': 3,
    'alpha': 1.,
    'capsize': 0,
    'capthick': 0,
    'x_label': 'Training step',
    'y_label': '',
    'legend_labels': ['Tunneling rate',
                      'Acceptance rate',
                      'Distance / step'],
    'title': title,
    'grid': True,
    'reverse_x': False,
}

### $(T = 1)$ Tunneling rate, Acceptance Rate and Avg. Distance vs. Training Step 

In [ ]:
%matplotlib notebook

In [ ]:
plt.rcParams['figure.facecolor'] = '#474747'

In [ ]:
out_file0 = out_file('tr_ar_dist_steps_lowT')#, step)
fig, axes = errorbar_plot(x_steps, y_data, y_err, out_file=out_file0, **kwargs)
_ = fix_legends(axes)
_ = fix_ticks(axes)
sfig = fig.savefig(out_file0, dpi=400, bbox_inches='tight')

### $(T > 1)$ Tunneling rate, Acceptance Rate and Avg. Distance vs. Training Step 

In [ ]:
# for trajectories with temperature > 1 vs. STEP
out_file1 = out_file('tr_ar_dist_steps_highT')#, step)
kwargs1 = kwargs.copy()
kwargs1['title'] = title_highT
fig, axes = errorbar_plot(x_steps, y_data_highT, y_err_highT,
              out_file=out_file1, **kwargs1)
axes = fix_legends(axes)
_ = fix_ticks(axes)
fig.savefig(out_file1, dpi=400, bbox_inches='tight')

### $(T = 1)$ Tunneling rate, Acceptance Rate and Avg. Distance vs. Temperature

In [ ]:
out_file2 = out_file('tr_ar_dist_temps_lowT')#, step)
# for trajectories with temperature = 1. vs TEMP
kwargs2 = kwargs.copy()
kwargs2['x_label'] = 'Temperature'
kwargs2['title'] = title
kwargs2['reverse_x'] = True
fig, axes = errorbar_plot(x_temps, y_data, y_err,
                          out_file=out_file2, **kwargs2)
_ = fix_legends(axes)
_ = fix_ticks(axes)
_ = add_vline(axes, 1, **{'color': 'C6', 'ls': '-', 'lw': 2.})
#axes[-1].set_xlim((15, 0.5))
fig.savefig(out_file2, dpi=400, bbox_inches='tight')
#ax.set_xlim(15, 1.05)

### $(T > 1)$ Tunneling rate, Acceptance Rate and Avg. Distance vs. Temperature

In [ ]:
out_file3 = out_file('tr_ar_dist_temps_highT')#, step)
# for trajectories with temperature > 1. vs TEMP
kwargs3 = kwargs.copy()
kwargs3['title'] = title_highT
kwargs3['x_label'] = 'Temperature'
kwargs3['reverse_x'] = True
fig, axes = errorbar_plot(x_temps, y_data_highT, y_err_highT,
                        out_file=out_file3, **kwargs3)
_ = add_vline(axes, 1, **{'color': 'C6', 'ls': '-', 'lw': 2.})
    
axes = fix_legends(axes)
_ = fix_ticks(axes)
fig.savefig(out_file3, dpi=400, bbox_inches='tight')
#axes[-1].set_xlim(10, 0.9)

### Annealing Schedule

In [ ]:
train_steps = data['_params']['num_training_steps']
temp_init = data['_params']['temp_init']
annealing_factor = data['_params']['annealing_factor']
annealing_steps = data['_params']['annealing_steps']
annealing_steps_init = data['_params']['_annealing_steps_init']
tunneling_steps = data['_params']['tunneling_rate_steps']
tunneling_steps_init = data['_params']['_tunneling_rate_steps_init']
#figs_dir = data['_params']['figs_dir']
#steps_arr = data['_params']['steps_arr']
#temp_arr = data['_params']['temp_arr']
#  num_steps = max(steps_arr)
max_steps_arr = max(steps_arr)
num_steps = max(train_steps, max_steps_arr)

#steps = np.arange(num_steps)
temps = []
steps = []
temp = temp_init
for step in range(num_steps):
    if step % annealing_steps_init == 0:
        tt = temp * annealing_factor
        if tt > 1:
            temp = tt
    if (step+1) % tunneling_steps_init == 0:
        steps.append(step+1)
        temps.append(temp)

In [ ]:
temp0 = data['_params']['temp_init']
#steps = np.arange(0, max(steps_arr))
#steps
annealing_factor = data['_params']['annealing_factor']
annealing_steps = data['_params']['annealing_steps']
tunneling_steps = data['_params']['tunneling_rate_steps']
fixed_temps = []
fixed_steps = []
temp = temp0
for step in range(max(steps_arr)):
    if step % annealing_steps == 0:
        tt  = temp * annealing_factor
        if tt > 1:
            temp = tt
    if (step+1) % tunneling_steps == 0:
        fixed_steps.append(step+1)
        fixed_temps.append(temp)

In [ ]:
plt.style.use('/Users/saforem2/.config/matplotlib/stylelib/dark_jupyter.mplstyle')
fig, ax = plt.subplots()
pt = ax.plot(steps, temps, ls='--', label='Fixed schedule', lw=2.5)
pt = ax.plot(steps_arr, temp_arr, label='Dynamic schedule', lw=2., alpha=0.75)
hl = ax.axhline(y=1., color='C6', ls='-', lw=2., label='T=1')
xl = ax.set_xlabel('Training step')
yl = ax.set_ylabel('Temperature')
lg = ax.legend(loc='best')
_ = fix_legends(ax)
#ylabels = ax.get_yticklabels()
#xlabels = ax.get_xticklabels()
#ax.set_yticklabels(ylabels, {'color': 'k'})
#ax.set_xticklabels(xlabels, {'color': 'k'})
#ax.set_yticklabels(ax.get_yticklabels(), {'color': 'k'})
#ax.set_xticklabels(ax.get_xticklabels(), {'color': 'k'})
_ = fix_ticks(ax)

plt.savefig(figs_dir + 'annealing_schedule.pdf', dpi=400, bbox_inches='tight')

In [ ]:
num_distributions = 4
sigma = 0.01
covs, distribution = gen_ring(r=1., var=sigma, nb_mixtures=num_distributions)
means = distribution.mus

In [ ]:
samples = distribution.get_samples(200)

In [ ]:
fig, ax = plt.subplots()
ax.plot(samples[:, 0], samples[:, 1], marker='o', ls='', alpha=0.75)
plt.show()

In [ ]:
for i in range(len(fixed_steps)):
    print(f'({fixed_steps[i]}, {fixed_temps[i]:.3g})\t'
          f'({steps_arr[i]}, {temp_arr[i]:.3g})')

In [ ]:
x_dim = 2
num_distributions = 2
sigma = 0.05
means = np.zeros((x_dim, x_dim))
rand_axis = np.random.randint(x_dim)
centers = 1

means[::2, :] = centers
means[1::2, :] = - centers
means = np.array(means).astype(np.float32)
cov_mtx = sigma * np.eye(x_dim).astype(np.float32)
covs = np.array([cov_mtx] * x_dim).astype(np.float32)
dist_arr = distribution_arr(x_dim, num_distributions)
distribution = GMM(means, covs, dist_arr)

In [ ]:
samples = distribution.get_samples(500)

In [ ]:
plt.style.use('/Users/saforem2/.config/matplotlib/'
               + 'stylelib/ggplot_sam.mplstyle')
fig, ax = plt.subplots()
ax.plot(samples[:,0], samples[:,1], marker='o', ls='', alpha=0.75)
fig.savefig('../log_mog_tf/run_22_diag_271/figures/diagonal_distributions_22.pdf', dpi=400, bbox_inches='tight')
plt.show()

In [ ]:
means = np.zeros((2, 2))
centers = 1.5
axis = 0
means[::2, axis] = centers
means[1::2, axis] = - centers

In [ ]:
sigma = 0.2
cov_mtx = sigma * np.eye(2)
covs = np.array([cov_mtx] * 2)

In [ ]:
means1 = np.zeros((2, 2))
centers1 = 1
means1[::2, axis] = centers1
means1[1::2, axis] = - centers1
sigma1 = 0.05
cov_mtx1 = sigma1 * np.eye(2)
covs1 = np.array([cov_mtx] * 2)

In [ ]:
from mog_model import distribution_arr

In [ ]:
dist_arr = distribution_arr(2, 2)

In [ ]:
distribution = GMM(means, covs, dist_arr)

In [ ]:
distribution1 = GMM(means1, covs1, dist_arr)

In [ ]:
samples = distribution.get_samples(500)

In [ ]:
samples1 = distribution1.get_samples(500)

In [ ]:
fig, ax = plt.subplots()
ax.plot(samples[:,0], samples[:,1], marker='o', ls='', alpha=0.75)
ax.plot(samples1[:,0], samples1[:,1], marker='o', ls='', alpha=0.75, color='C1')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(samples[:,0], samples[:,1], marker='o', ls='', alpha=0.75)
ax.plot(samples1[:,0], samples1[:,1], marker='o', ls='', alpha=0.75, color='C1')
plt.show()